## `mict` -- middle-ground between `dict` and a `class`

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jerzydziewierz/mict/HEAD?filepath=demo.ipynb)

Provides `MATLAB`-struct-like dot notation for setting and of items in a `dict` (dictionary), and a handful of interactivity tools.

`mict` is intended to be a middle ground between `dict` and full fledged `class` / `object` pattern for structured data storage.

It does a bit more than basic `dict`, but does not attempt to supersede pandas, numpy nor other advanced data storage tools


## Installation

try installing with `pip`, grabbing it straight from git:

```bash
python -m pip install -e -U -q git+https://github.com/jerzydziewierz/mict.git#egg=mict
```

## Demo

Online demo with binder: 

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/jerzydziewierz/mict/HEAD?filepath=demo.ipynb)

local demo: please see `demo.ipynb` -- a jupyter notebook.

## Basic usage

In [1]:
from mict import mict
q=mict()
q.first = 'Hello world!'
q

key,value
first,Hello world!


---

the key power of mict is the ease of adding, removing, altering the contents -- and the the nice visualisation of it's contents. 

You can add new keys easily, and then access them using dot-notation:

In [2]:
q.third = 3
q

key,value
first,Hello world!
third,3


You can store functions and call them. `mict` attempts to display the signature of the function for you. 

In [3]:
def _some_fn(self,y):
    return self.third+y 
q.add = _some_fn 
q.sub = lambda self,y: self.third-y
q

key,value
first,Hello world!
third,3
add,"add(self,y)"
sub,"sub(self,y)"


call the function from inside the `mict`, that operates on that mict:

In [4]:
q.add(q,2)

5

In [5]:
q.sub(q,2)

1

--- 

The visualiser function in `mict` is called `reprstyler`.

A new `mict` instance comes with the `reprstyler_html` already set to `reprstyler_basic_html` but you can, and should make your own reprstylers.

example reprstyler:

## Convinence save and load functions using pickle

> **Warning**: Since the `pickle` save/load also loads executable functions (including the `reprstyler`), all the precautions that apply to `pickle` apply here. Loading untrusted files is potentially unsafe.

methods `to_pickle` and `from_pickle` work as you would expect:

In [6]:
q.to_pickle('demo.pickle')

True

In [7]:
q=mict.from_pickle('demo.pickle')
q

key,value
first,Hello world!
third,3
add,"add(self,y)"
sub,"sub(self,y)"


## Customizing the reprstyler 

This is the real reason why I developed mict: I wanted to have a quick and simple way to customize how the contents of the dictionary are visualized.
    

In [8]:
from mict import mict
def mickiestyler(self):
    return f'mickie score: {self.mickie:0.3f}'

q=mict(zap=None,mickie=6.28,hidden_value1=45)
q

key,value
zap,None
mickie,6.28
hidden_value1,45


In [9]:
q.reprstyler_html = mickiestyler
q

{'zap': None, 'mickie': 6.28, 'hidden_value1': 45, 'reprstyler_html': <function mickiestyler at 0x7fddbc84b430>}

In [10]:
q.reprstyler_html = None
q

{'zap': None, 'mickie': 6.28, 'hidden_value1': 45, 'reprstyler_html': None}

---

Even tough you should strive to make all your objects functional and immutable in general, for development purposes it is convenient to mutate your objects sometimes. The reprstyler can be changed **after** initializing the `mict` storage itself

In [11]:
def new_reprstyler(subject=None):
    txt = 'keys:  '
    for key in subject.keys():
        txt = f"{txt} {key} ={subject[key]};"
    return txt

q.reprstyler_html = new_reprstyler
q

{'zap': None, 'mickie': 6.28, 'hidden_value1': 45, 'reprstyler_html': <function new_reprstyler at 0x7fddbc7d5040>}

then, if you want a different styler for jupyter, and a different one for text-only, you can (optionally) override the `reprstyler` with `reprstyler_html`. It will only be used if `_repr_html_` is called -- as `jupyter` first tries to call that first.

In the examples below, note how reprstyler is executed to produce html based on the current contents of the `mict`


In [12]:
def mickiestyler_html(subject):
    return f'<h4>✅mickie score: {subject.mickie:0.3f}</h4>' if subject.mickie>5 else f'<h4>❌not mickie enough: {subject.mickie:0.3f}</h4>'
q.reprstyler_html = mickiestyler_html
q

{'zap': None, 'mickie': 6.28, 'hidden_value1': 45, 'reprstyler_html': <function mickiestyler_html at 0x7fddbc7d5430>}

In [13]:
q.mickie = 3
q

{'zap': None, 'mickie': 3, 'hidden_value1': 45, 'reprstyler_html': <function mickiestyler_html at 0x7fddbc7d5430>}

In [14]:
q.reprstyler_html

<function __main__.mickiestyler_html(subject)>

## An even quicker way to make a reprstyler

You can use `lambda` anonymous function definition to make a reprstyler quickly.

Note that reprstyler_html will display any html you give it, including images, videos, sound or javascript inserts

In [15]:
q=mict(fill='blue',r=15)
q.reprstyler_html= lambda this:f'<svg height="100" width="100"><circle cx="50" cy="50" r="{this.r}" stroke="black" stroke-width="3" fill="{this.fill}" /></svg>'
q


{'fill': 'blue', 'r': 15, 'reprstyler_html': <function <lambda> at 0x7fddbc7d5dc0>}

In [16]:
q.r=32
q.fill='red'
q

{'fill': 'red', 'r': 32, 'reprstyler_html': <function <lambda> at 0x7fddbc7d5dc0>}

Remove the `reprstyler` and use the default `dict` styler.

use the `tpop` method which is equivalent to `try pop(); except pass` - hence does not throw exception on no key present

In [17]:

q.reprstyler_html=None
q.tpop('reprstyler_html')

{'fill': 'red', 'reprstyler_html': None}

In [18]:
q

{'fill': 'red', 'reprstyler_html': None}

---

## Note

If you really feel like it, this could be extended to markdown, png, svg and other visualizers supported by jupyter.


Importantly, you can still see the classic `dict` __repr__ function (lists all keys/values) using

```python
super(handybeam.dict,q).__repr__()
```

# Advanced uses

## Show shape of big variables, instead of it's content.

many of the actual research code will use numpy arrays or long lists. These are typically unwieldy to just display.

`mict` provides a way to only display the shape of the numpy/tensorflow/pytorch/jax array, instead of the contents. You will find that this is often what you need displayed.

In [19]:
from mict import reprstyler_basic_html
import numpy
q=mict(small_array=numpy.array([1,2,3,4,5]))
q

key,value
small_array,"np.array(shape=(5,))"


In [20]:
q.large_array=numpy.random.random((150,150))
q.reprstyler_html = reprstyler_basic_html # optional set -- this is already a default
q

key,value
small_array,"np.array(shape=(5,))"
large_array,"np.array(shape=(150, 150))"


# Advanced uses

## Capture locals from inside the function and return them in a `dict` / `mict` 

When developing a research function, you will often want to capture **all** the locals inside it for debug purposes. 

Only when stabilizing the implementation, you will want to prune the result and leave usefull return values only.

In [21]:
 # define a function
def do_maths(x=1,y=2):
    a=x+y
    b=a*x
    c=b*y
    result = mict.from_locals()  # puts `x`,`y`,`a`,`b`,`c` into `result`. 
    result.pop('b') # remove 'b' from the result
    return result

# execute that function
demo_result = do_maths(x=4,y=2)

# review function's locals
demo_result

key,value
x,4
y,2
a,6
c,48


> Danger zone : this will overwrite your locals, and may lead to crazy results if something crazy was stored in the mict.

load the variables from the mict back into locals.

In [22]:
demo_result.unpack_to_locals()
x

4

In [23]:
y

2

# Advanced uses

## Nested dictionary use and special attributes

note that `name` and `type` have a special meaning for the default visualizer, `reprstyler_basic_html`.

Moreover, the `reprstyler_basic_html` will try to obtain html from inner (nested) `mict`s and display it in a table.

Note that `reprstyler_basic_html` is merely another function from the `mict` module, and can be overridden with any other reprstyler. If `reprstyler_html` is set to `None`, the default `dict.__repr__()` is used.

In [24]:
x=mict(name='first',type='coordinate', value=3)
y=mict(name='second', type='coordinate', value=5)
point=mict(type='outer <b>mict</b>',x=x,y=y)

In [25]:
x

key,value
value,3


In [26]:
y

key,value
value,5


In [27]:
point

{'type': 'outer <b>mict</b>', 'x': {'name': 'first', 'type': 'coordinate', 'value': 3}, 'y': {'name': 'second', 'type': 'coordinate', 'value': 5}}

In [28]:
v=point._repr_html_()
v

'<em>Type:</em> outer <b>mict</b>; <br/><table><tr><th>key</th><th>value</th></tr><tr><td>x</td><td><em>Type:</em> coordinate; <em>Name:</em> first; <br/><table><tr><th>key</th><th>value</th></tr><tr><td>value</td> <td> 3</td> </tr>  </table></td> </tr> <tr><td>y</td><td><em>Type:</em> coordinate; <em>Name:</em> second; <br/><table><tr><th>key</th><th>value</th></tr><tr><td>value</td> <td> 5</td> </tr>  </table></td> </tr>  </table>'

In [29]:
point.reprstyler_html=None

In [30]:
point

{'type': 'outer <b>mict</b>', 'x': {'name': 'first', 'type': 'coordinate', 'value': 3}, 'y': {'name': 'second', 'type': 'coordinate', 'value': 5}, 'reprstyler_html': None}

In [31]:
isinstance(point,mict)

True

## More examples

In [32]:
from mict import mict
import numpy
import math
q=mict(title="some title",subtitle="some subtitle",interesting_integer = 3,interesting_float = math.tau,  big_array=numpy.random.random((200,250)))
q

key,value
title,some title
subtitle,some subtitle
interesting_integer,3
interesting_float,6.28
big_array,"np.array(shape=(200, 250))"


In [60]:
def custom_html_styler(this):
    output = f'<h1>{this.title}</h1>'
    output = f'{output}<h2>{this.subtitle}</h2>'
    output = f'{output}<p>interesting integer:{this.interesting_integer:04d}</p>'
    output = f'{output}<p>interesting float:{this.interesting_float:0.{this.interesting_integer}f}</p>'
    output = f'{output}<p>some stats: {this.big_array.std()=:0.4f}</p>'
    output = f'{output}<hr/>'
    output = f"""{output}<button id="bt1" onclick="alert('any javascript goes.')" style="width:500px; margin-left:200px">raise alarm</button>"""
    output = f'{output}<hr/>'
    return output

q.reprstyler_html = custom_html_styler
q

{'title': 'some title', 'subtitle': 'some subtitle', 'interesting_integer': 3, 'interesting_float': 6.283185307179586, 'big_array': array([[0.56010521, 0.72958158, 0.1902957 , ..., 0.01983149, 0.54744355,
        0.91040881],
       [0.4044862 , 0.46932414, 0.7893918 , ..., 0.91853963, 0.28227311,
        0.9563306 ],
       [0.02124869, 0.2343184 , 0.13481907, ..., 0.04111261, 0.24252155,
        0.71227556],
       ...,
       [0.90988761, 0.23970703, 0.25774679, ..., 0.87997978, 0.99437305,
        0.39329894],
       [0.6432976 , 0.72931159, 0.88774214, ..., 0.47381733, 0.40311719,
        0.39590683],
       [0.09017084, 0.72583605, 0.5375708 , ..., 0.13186973, 0.13164289,
        0.94042024]]), 'reprstyler_html': <function custom_html_styler at 0x7fddbb1d49d0>}

In [61]:
q.big_array.std()

0.2888494364193682

## Gotchas

* `mict` does not throw an error when trying to access undefined field. Instead, it gives a warning and returns `None`


In [62]:
type(q.this_key_does_not_exist)

/tmp/ipykernel_69670/3203445190.py:1: UserWarning: no key "this_key_does_not_exist" in this dictionary. 
valid keys are ['title', 'subtitle', 'interesting_integer', 'interesting_float', 'big_array', 'reprstyler_html']
  type(q.this_key_does_not_exist)
/home/mib07150/miniconda3/envs/cutfm21/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3441: UserWarning: in 
  exec(code_obj, self.user_global_ns, self.user_ns)
/home/mib07150/miniconda3/envs/cutfm21/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3361: UserWarning: in 
  if (await self.run_code(code, result,  async_=asy)):


NoneType

--- 

* `dict` keys that contain a dot, cannot be used to access the field in dot-notation mode.

For example:


In [63]:
q=mict()
q['a.b']=3
q.a.b  # UserWarning and AttributeError - no key "a" in this dictionary.

/tmp/ipykernel_69670/3975629951.py:3: UserWarning: no key "a" in this dictionary. 
valid keys are ['a.b']
  q.a.b  # UserWarning and AttributeError - no key "a" in this dictionary.


AttributeError: 'NoneType' object has no attribute 'b'




---

## Attributions

Happily copypasted from https://stackoverflow.com/questions/2352181/how-to-use-a-dot-to-access-members-of-dictionary , and modified only slightly.

then, extended a bit.

then, a bit more, with optional reprstyler. See the source code for `self.__repr__()`.

then, extended again with advanced default reprstyler, `from_locals()`, `unpack_to_locals()`, `to_pickle()`, `from_pickle()` and `tpop()`. I am now using `mict` nearly daily and i feel it improves my results.


## Related packages 

see also: 

https://pypi.org/project/python-box/

https://github.com/drgrib/dotmap



## License

MIT License, Copyright (c) 2019-2021, George Rey, Paris. See LICENSE file.
